In [3]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import time
import re
import random
driver = webdriver.Chrome(executable_path='C:\chromedriver\chromedriver.exe')


In [271]:
##HELPERFUNCTIONS

def get_matchLinks(page):
    matchesLinks = []
    ## https://www.hltv.org/results?offset=100&startDate=all&content=stats&stars=1
    driver.get('https://www.hltv.org/results?offset='+ str(page) +'&startDate=all&content=stats&stars=1')
    content = driver.page_source
    soup = BeautifulSoup(content)
    
    ## FIND MATCHES
    for div in soup.findAll('div', attrs={'class': 'results'}):
        for a in div.findAll('a', attrs={'class': 'a-reset'}):
            link = a['href']
            if (link[:8] == "/matches"):
                matchesLinks.append(link)
    return matchesLinks

def get_page_data(link):
    url = 'https://www.hltv.org/' + str(link)
    driver.get(url)
    content = driver.page_source
    soup = BeautifulSoup(content)    
    return soup

def get_game_links(soup):
    statlinks = []
    for div in soup.findAll('div', attrs={'class': 'results-center-stats'}):
        for a in div.findAll('a'):
            statlinks.append(a['href'])
    return statlinks

def get_game_stats(link):
    result = get_page_data(link)
    return result

In [272]:
## GAME STAT INFO
##GATHER PLAYER STATS

#print(data[8][0])
#LINK /stats/matches/mapstatsid/118030/astralis-vs-og
def gather_player_stats(link):
    colnames = ["Player","K (hs)","A (f)","D","KAST","K-D Diff","ADR","FK Diff","Rating","Team"]

    #localLink = data[8][0]
    result = get_page_data(link)
    page_source = result
    result = result.findAll('table', {'class':'stats-table'})
    results = pd.read_html(str(result))

    MapID = link.split("/")[4]

    #print(MapID)
    #print(Map)
    df_team1 = pd.DataFrame(results[0])
    df_team1['Team'] = df_team1.columns[0]
    df_team1.columns = colnames
    df_team2 = pd.DataFrame(results[1])
    df_team2['Team'] = df_team2.columns[0]
    df_team2.columns = colnames

    game_df = df_team1.append(df_team2,ignore_index=True)
    game_df['MapID'] = MapID


    misc_stats = page_source.findAll('div', attrs={'class': 'right'})
    game_df['Starting side'] = misc_stats[0].contents[4]['class'][0][:-6]
    game_df['Rounds'] = misc_stats[0].text
    game_df['Team rating'] = misc_stats[1].text
    game_df['Entries'] = misc_stats[2].text
    game_df['Clutches'] = misc_stats[3].text
    #game_df['Map'] 
    #print(game_df)
    #game_df.to_csv(index=True)
    return game_df


In [273]:
##BASIC INFO

def gather_match_stats(links):
    match_df = pd.DataFrame()
    ##GET DATE from https://www.hltv.org/matches/2347834***
    names = ["Date","Tournament","Team1","Result1","Team2","Result2","Map","MapID","StatLinks"]
    data = []
    maps = []
    mapsID = []
    pick = []

     
    choiceslist = soup.find_all('div',class_="standard-box veto-box")
    choices = (str(choiceslist[1].contents[1].text).strip("\n").strip(" "))
    #print(choices)


    for div in soup.findAll('div', attrs={'class': 'match-page'}):
        pageDate = div.find('div', attrs={'class': 'date'})
        pageTournament = div.find('div', attrs={'class': 'event text-ellipsis'})
        data.append(pageDate.text)
        data.append(pageTournament.text)

    ##GET FINALRESULT1 from https://www.hltv.org/matches/2347834***
    for div in soup.findAll('div', attrs={'class': 'team1-gradient'}):
        pageTeam1 = div.find('div', attrs={'class': 'teamName'})
        pageResult1 = div.find('div', attrs={'class': ['won', 'lost', 'tie']})
        data.append(pageTeam1.text)
        data.append(pageResult1.text)

    ##GET FINALRESULT2 from https://www.hltv.org/matches/2347834***
    for div in soup.findAll('div', attrs={'class': 'team2-gradient'}):
        pageTeam2 = div.find('div', attrs={'class': 'teamName'})
        pageResult2 = div.find('div', attrs={'class': ['won', 'lost', 'tie']})
        data.append(pageTeam2.text)
        data.append(pageResult2.text)

    ##GET MAPS AND MAPSID from https://www.hltv.org/matches/2347834***
    for div in soup.findAll('div', attrs={'class': 'stats-menu-link'}):
        map1 = div.findAll('div', attrs= {'class': 'dynamic-map-name-full'})
        for div in map1:
            maps.append(div.text)
            mapsID.append(div['id'])

    ##GET BETTING ODDS
    bets = []
    try:
        for div in soup.findAll('div', attrs={'class': 'past-matches-grid'}):
            for bet in div.findAll('div', attrs={'class': 'past-matches-bottom-right-numbers'}):
                bets.append(bet.text)
    except:
        pass
    
    #get_map_picks
    picks = get_map_picks(soup)

    
    data.append(maps)
    data.append(mapsID)
    #data.append(links)


    ##GET DETAILED STATS LINKS
    
    #----------
    ## FOR EACH MAP in
    for i in range(1,len(maps)):
        #print(maps[i])
        #print(mapsID[i])
        #print(links[i-1])
        game_df = gather_player_stats(links[i-1])
        game_df['Map'] = maps[i]
        game_df['Tournament'] = data[1]
        game_df['Date'] = data[0]
        game_df['Team1'] = data[2]
        game_df['Team2'] = data[4]
        game_df['ResultTeam1'] = data[3]
        game_df['ResultTeam2'] = data[5]
        if maps[i] in picks:
            game_df['T1pick'] = picks[maps[i]]
        try:
            game_df['Bets'] = str(bets[0]) + " / " + str(bets[1])
        except:
            game_df['Bets'] = ""

        match_df = match_df.append(game_df)
    
    #print(choices)
    match_df['choices'] = str(choices)

    return match_df
    #match_df.to_clipboard()



    #data_df.iloc[7].values[0][0] -> maps
    #data_df.iloc[8].values[0][0] -> mapID
    #data_df.iloc[9].values[0][0] -> players

In [274]:
##MAIN
for j in range(0,100):   
    print("J: "+str(j))
    total_matches = 0
    n_runs = 100
    runname = "CSGO_Data_bets_picks_choices"


    #df_running = pd.read_csv(str(runname+'.csv'))
    #total_matches = len(df_running['MatchID'].unique())
    
    print("Stating at: "+ str(total_matches)+ " with " + str(n_runs) + " incs")
    

    #CAREFUL FETCHES PAGE (runtime=1)
    matchesLinks = get_matchLinks(total_matches) #goes to results page to fetch matchLinks
    #print(pd.DataFrame(matchesLinks))

    for i in range(0,n_runs): #len(matchesLinks)
        match_id = matchesLinks[i].split("/")[2]
        if len(df_running['MatchID'].where(df_running['MatchID'] == match_id).dropna()) != 0:
            print("Duplicate Skipping" + match_id)
        else:
            print("Fetching: " +str(match_id) + " I: "+ str(i))
            #CAREFUL FETCHES PAGE
            soup = get_page_data(matchesLinks[i]) #gets the soup for matchpage from match links
            gamelinks = get_game_links(soup) #gets the gamelinks from match links
            output = gather_match_stats(gamelinks)  #gets stats from gamelinks
            output['MatchID'] = match_id
            #print(output)
            output.to_csv(str(runname)+'.csv', mode='a', header=False,index=False)



J: 0
Stating at: 0 with 1 incs
Fetching: 2347745 I: 0


In [43]:
##
bets = []
for div in soup.findAll('div', attrs={'class': 'past-matches-grid'}):
    for bet in div.findAll('div', attrs={'class': 'past-matches-bottom-right-numbers'}):
        bets.append(bet)

1.44
2.62
1.44
2.62


In [223]:
df_running = pd.read_csv("RunningTest.csv")
len(df_running['MatchID'].where(df_running['MatchID'] == match_id).dropna())

0

In [186]:
matchesLinks_ids = []
for i in range(0,len(matchesLinks)):
    matchesLinks_ids.append(matchesLinks[i].split("/")[2])
print(len(matchesLinks[]))

54


In [204]:
df_running = pd.read_csv("run_Test_50.csv")
print(len(df_running['MatchID'].unique()))

48


for i in range(0,len(results)):
    print(results[i])
df = pd.DataFrame(result[0])
print(df)

In [162]:
soup = get_page_data("/matches/2347840/izako-boars-vs-blink-esea-advanced-season-36-europe") #gets the soup for matchpage from match links
print(soup.prettify())

 no-promode" href="/forums/threads/2444688/do-you-still-respect-germany">
          <span class="topic a-default" title="Cs: do you still respect germany">
           do you still respect germany
          </span>
          197
         </a>
         <a class="col-box activity a-reset csCat no-promode" href="/forums/threads/2444779/ex-tuureboelius-come-here">
          <span class="topic a-default" title="Cs: ex-TuureBoelius come here">
           ex-TuureBoelius come here
          </span>
          22
         </a>
         <a class="col-box activity a-reset newsCat" href="/news/31550/nafany-we-want-to-establish-our-own-era">
          <span class="topic a-default" title='News: nafany: "We want to establish our own era"'>
           nafany: "We want to establish our own era"
          </span>
          225
         </a>
         <a class="col-box activity a-reset csCat no-promode" href="/forums/threads/2444685/your-son-name">
          <span class="topic a-default" title="Cs: Your so

In [88]:
for div in soup.find_all('div',class_="pick"):
    if "results-left" not in div:
        print("Team1Picked")
    else:
        print("Team2Picked")

for span in soup.find_all('span',class_="pick"):
        if "results-right" not in span:
            print("Team2Picked")
        else:
            print("Team1Picked")

Team1Picked
Team1Picked
Team2Picked
Team2Picked


In [269]:
#FETCH PICK


def get_map_picks(soup):
    item = soup.find(True,class_="flexbox-column")
    pick_dict = {}
    for game in item.find_all(True,class_="mapholder"):
        #print(game.contents[1].contents[1].text)
        if 'pick' in str(game.contents[3].contents[1].attrs):
            #print("T1_pick")
            pick_dict[game.contents[1].contents[1].text.strip("\n")] = '1'
        elif 'pick' in str(game.contents[3].contents[5].attrs):
            #print("T2_pick")
            pick_dict[game.contents[1].contents[1].text.strip("\n")] = '0'
        else:
            #print("error")
            pick_dict[game.contents[1].contents[1].text.strip("\n")] = 'NA'
    return pick_dict

    #print(game.contents)
    #print("###")
    
    #print(game.contents[3].contents[5].attrs)
    #print("-------------------")
    #print(game.contents[3].contents[1])
    



In [222]:

print(



1. MIBR removed Nuke
2. FURIA removed Dust2
3. MIBR picked Mirage
4. FURIA picked Vertigo
5. MIBR removed Inferno
6. FURIA removed Overpass
7. Train was left over


